# Sprint 2 - Basic Security, Acid transactions

## TODO:

### Begin CLI (low priority)

\+ NOTE: WORK ON THE UVIC CS SERVER

- Difficult to do allocation at this time, since program is nonexistent
- For *this* sprint, attempt to get the beginning of a working program, i.e. command line interface
  - Generates sql database
  - Asks for tickers
  - Get data for tickers from yf
  - Populate database with stock data
- *Next* sprint, we will make another module which creates the allocation.

Important:
- Write down exactly what commands were run to create sql server
  - These need to be in the readme so that the user can access the program
  - Probably the user will have to create their own server; including step-by-step in hte readme will be good practice and allow the program to be universally usable
  - Add functionality so that program asks for username, password, database name and sets corresponding variables in main.py

### Course work

#### Last week:

- [ ] Complex sql queries with relational algebra

#### This week:

- [x] Sub-queries
- [x] Using indexes
- [ ] Basic security; managing access (integrate into CLI/db setup)
- [ ] Understanding views

### Work

#### Andrew

- [ ] Get fetch data.py to output correct file; populate 'stock' and 'history'

#### Evan

- [ ] Edit main file to produce desired behavior; populate 'session' and 'portfolio'
- [ ] Catch-up notes: Complex sql, security
- [ ] Ask about theoreticlal justification

# Notes: Security

- Should be setting up permissions immediately, 'user' defined as someone accessing DB through the CLI

- Idea:
  - Code which creates database requires some kind of keyword or user change in order to run db commands
  - DB commands will NOT be run through command lineCode which creates database requires some kind of keyword or user change in order to run db commands
  - Make DB initialization so that the user is given next to no privileges off the bat - must switch to a 

## CLI design

What does it need to do?
- [ ] Sign in to a premade database 
  - See if there's a possibility of automated setup

GetHistory.py

import yfinance as yf
from datetime import datetime, timedelta

# Function to fetch stock history
def get_history(ticker, start_date, end_date):
    try:
        # Fetch stock history for the past week
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date)
        
        # Extract relevant data (date and price)
        history_data = []
        for date, row in hist.iterrows():
            history_data.append((ticker, date.strftime('%Y-%m-%d'), row['Close']))
        
        return history_data
    except Exception as e:
        print(f"Error fetching history for {ticker}: {e}")
        return []

def main(tickers=""):
    if not tickers:
        tickers = input("Enter the stock ticker symbols (separated by commas): ")

    tickers = tickers.split(',')

    # Get current date and date one week ago
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)

    # Initialize history_id counter
    history_id = 1

    # List to hold the fetched data
    history_data_list = []

    # Fetch and store stock history data for each ticker symbol
    for ticker in tickers:
        ticker = ticker.strip()
        history_data = get_history(ticker, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        for data in history_data:
            history_data_list.append((history_id,) + data)
            history_id += 1

    # Prepare the SQL insert statements
    insert_statements = []
    for data in history_data_list:
        hid, ticker, date, price = data
        insert_statement = f"INSERT INTO history (historyID, Ticker, Date, Price) VALUES ({hid}, '{ticker}', '{date}', {price});"
        insert_statements.append(insert_statement)

    # Combine all insert statements into a single string
    sql_insert_statements = "\n".join(insert_statements)

    # Save the SQL insert statements to a file
    with open("insert_history_statements.sql", "w") as file:
        file.write(sql_insert_statements)

    return sql_insert_statements

# Example usage
if __name__ == "__main__":
    tickers = "AAPL, MSFT, GOOGL"
    print(main(tickers))


GetStock.py

In [ ]:
import yfinance as yf

# Function to fetch stock data
def get_stock(ticker, stock_id):
    try:
        # Create a Ticker object for the specified ticker
        stock = yf.Ticker(ticker)
        
        # Fetch stock info with safe default values
        stock_info = stock.info
        symbol = stock_info.get('symbol', ticker)  # Default to the input ticker if 'symbol' is not available
        sector = stock_info.get('sector', 'Unknown')  # Default to 'Unknown' if 'sector' is not available
        price = stock_info.get('regularMarketPreviousClose', 0.0)  # Default to 0.0 if 'regularMarketPreviousClose' is not available
        sd = stock_info.get('beta', 0.0)  # Default to 0.0 if 'beta' is not available
        eret = round((stock_info.get('forwardEps', 0.0) / price) * 100, 3)  # Default to 0.0 if 'forwardEps' is not available
        
        return stock_id, symbol, sector, price, sd, eret
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return stock_id, None, None, None, None, None

def main(tickers = ""):
    if tickers == "":
        tickers = input("Enter the stock ticker symbols (separated by commas): ").split(',')

    tickers = tickers.split(',')

    # Initialize stock_id counter
    stock_id = 1

    # List to hold the fetched data
    stock_data_list = []

    # Fetch and store stock data for each ticker symbol
    for ticker in tickers:
        ticker = ticker.strip()  # Remove any leading/trailing whitespace
        stock_data = get_stock(ticker, stock_id)
        if stock_data[1]:  # Check if symbol is not None
            stock_data_list.append(stock_data)
        stock_id += 1

    # Prepare the SQL insert statements
    insert_statements = []
    for data in stock_data_list:
        stock_id, symbol, sector, price, sd, eret = data
        insert_statement = f"INSERT INTO stocks (StockID, Ticker, Sector, Price, SD, ERet) VALUES ({stock_id}, '{symbol}', '{sector}', {price}, {sd}, {eret});"
        insert_statements.append(insert_statement)

    # Combine all insert statements into a single string
    sql_insert_statements = "\n".join(insert_statements)

    # Save the SQL insert statements to a file
    # with open("insert_statements.sql", "w") as file:
    #     file.write(sql_insert_statements)

    
    return sql_insert_statements

# if __name__ == "__main__":
#     tickers = input("Enter the stock ticker symbols (separated by commas): ").split(',')
#     main(tickers)


### Course level competencies: Data analytics

Complete:
- Level 1
- (L2, 1) Expresses complex logic as single SQL queries using aggregation and sub-queries
- (L2, 2) Understands how functional dependencies and referential integrity affect the semantics of queries
- (L2, 3) Describes the logical ordering of operators in complex queries that involve nested logic

Moving forwards:
- (L2, 4) Plans out how to transform data from relations into a desired output layout as in standard OLAP/ETL operators
- (L2, 5) Prefers embedding complex logic into RDBMS over handling it in application-layer code

## Goals from previous sprint

### Data analytics

Attempt to 
- (2), (4) Perform various complex sql queries with relational operators, including JOIN operations, next sprint

### Data Modelling

Attempt to (level 2)
- (3) Begin theoretical justification: Uncertain about the level of justification required, this is a question for instructor/TA
- (5) Internalize (perhaps outline) merits of the relational data model (?)

### Back-end engineering

Attempt to:
- Understand what is expected for level 1, use this understanding to inform the upcoming sprint

### Summary

- Complex sql queries with relational algebra
- Ask about theoretical justification 
- Begin to implement back-end engineering principles that are covered in the coming weeks